In [ ]:
cd /Users/XX/ZoRaFa/yolov5_train_test
# Replace the XX in file paths. 

In [ ]:
pip install -r requirements.txt

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
from pycocotools.coco import COCO
import pandas as pd
import random
from PIL import Image, ImageDraw, ImageFont, ExifTags
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.patches as patches
from matplotlib.pyplot import figure
from shapely.geometry import Polygon
import pylab
pylab.rcParams['figure.figsize'] = (8.0, 10.0)
import cv2
import os
import datetime
import torch
import torchvision

we have that the 
- coco : common objects in context, a way to do object detection using json sstructuure which determines how the labels are annotated
- pycocotools: helps doig the parsing ...

## Load Data
### CSV Files from SharePoints (Images/BBOXES)

In [ ]:
cd /Users/XX/Documents/surfrider/surfnet/data/images/annotations

In [ ]:
df_bboxes = pd.read_csv("bounding_boxes_202205231416.csv")
df_images = pd.read_csv("images_for_labelling_202205231708.csv")

In [ ]:
df_images.head()

In [ ]:
print(f'Nb. of bboxes : {len(df_bboxes)}')
print(f'Nb. of images : {len(df_images)}')

I believe we have that there is the images; with different caracteristics, such as the type of view that there is, the nagle of the pic, the quality and the context - the water source/type 

Then there is the bonding box, each one is linked to an image ; and has 4 coordinates which determine sthe box. There is also the class ID : with the type of pollution detected (0) -10 

### Instances Files

In [ ]:
coco_train  = COCO(annotation_file = 'instances_train.json')
coco_valid  = COCO(annotation_file = 'instances_val.json')
coco_images = COCO(annotation_file = 'instances.json')

coco_images_train = coco_train.dataset['images'][1:]
coco_images_valid = coco_valid.dataset['images'][1:]
coco_images       = coco_images.dataset['images'][1:]

Here we have that the COCO loads the annotation file and prepare data structure

In [ ]:
print(f'Nb. of train images : {len(coco_images_train)}')
print(f'Nb. of val.  images : {len(coco_images_valid)}')
print(f'Nb. of total images : {len(coco_images)}')

In [ ]:
IMAGES_PATH = "images2label"
images      = os.listdir(IMAGES_PATH)
images_full_path = [os.path.join(IMAGES_PATH, x) for x in os.listdir(IMAGES_PATH)]

print(f'Image Path : {len(images)}')

## Data Exploration

In [ ]:
filenames = list(df_images.filename.unique())
len(filenames)
# here we're taking the files that are unique : so got rid of the doubles

In [ ]:
filenames_train = [obj['file_name'] for obj in coco_images_train]
filenames_valid = [obj['file_name'] for obj in coco_images_valid]
print("TRAIN", len(filenames_train), "VALID", len(filenames_valid))
filenames_train_valid = filenames_train + filenames_valid
print("TOTAL", len(filenames_train_valid))

#here we have the total train and validity is of 3921 and then the rest would be the one who 
#are on the csv file but not on the instances
# we have that we divided our images into 2 sets one for training and one for validation 

### Files included in CSV and Instances

In [ ]:
files_inter = list(set(filenames) & set(filenames_train_valid))
len(files_inter)

### Files that are CSV but not instances 

In [ ]:
set_difference_f = set(filenames) - set(filenames_train_valid)
list_difference_f = list(set_difference_f)
len(list_difference_f)

### Names of the files of the Instances that are not in CSV

In [ ]:
set_difference_i = set(filenames_train_valid) - set(filenames)
list_difference_i = list(set_difference_i)
len(list_difference_i)

### GroupBy ID

In [ ]:
cd /Users/XX/ZoRaFa/yolov5_train_test/src/surfnet_train/visualization

In [ ]:
from categories import annotations_cat
annotations_cat(df_bboxes);

the data is unbalanced - way more of categpry 1 : sheet, tarp, plastic bags and fragment 
- Sheet / tarp / plastic bag / fragment
- Insulating material
- Bottle-shaped
- Can-shaped
- Drum
- Other packaging
- Tire
- Fishing net / cord
- Easily namable
- Unclear

### GroupBy Context

In [ ]:
from categories import img_context
img_context(df_images);

The data is unbalanced : once again much more of the river then others ; small beach

### GroupBy View

In [ ]:
from categories import img_view
img_view(df_images);

### GroupBy Image quality

In [ ]:
from categories import img_quality
img_quality(df_images);

## Data Processing

In [ ]:
cd /Users/XX/ZoRaFa/yolov5_train_test/src/surfnet_train/data

In [ ]:
from data_processing import coco2yolo
from data_processing import get_df_train_val

In [ ]:
!mkdir ./images
!mkdir ./labels

In [ ]:
cd /Users/XX/ZoRaFa/yolov5_train_test/src/surfnet_train/models 

In [ ]:
import path_existance

In [ ]:
get_df_train_val("/Users/XX/Documents/surfrider/surfnet/data/images/annotations/instances.json", "/Users/XX/Documents/surfrider/surfnet/data/images/annotations/images2label" , df_images)

### Get Categories

In [ ]:
annotation_file = "annotations-surfnet/instances_train.json"
coco = COCO(annotation_file)

coco_categories = coco.dataset['categories'][1:]
class_id_to_name_mapping = {}
for c in coco_categories:
    class_id_to_name_mapping[c['id']-1] = c['name']

class_id_to_name_mapping

### Creation of 2 DataFrames for future manipulation 5GrouppKFold, KFold, Split)

In [ ]:
data_dir = "images2label"

def get_df_train_val(annotation_file):
    
    coco = COCO(annotation_file) #transform the file 

    old_filenames  = []
    dates          = []
    views          = []
    images_quality = []
    contexts       = []
    all_bboxes     = []
    all_images     = []
    new_filenames  = []
    new_labelnames = []

    img_ids = np.array(coco.getImgIds())
    
    for img_id in img_ids:
        image_infos = coco.loadImgs(ids=[img_id])[0]

        if os.path.exists(os.path.join(data_dir, image_infos['file_name'])):

            date_creation  = df_images.loc[df_images["filename"] == image_infos["file_name"]]["createdon"].values[0]
            view           = df_images.loc[df_images["filename"] == image_infos["file_name"]]["view"].values[0]
            image_quality  = df_images.loc[df_images["filename"] == image_infos["file_name"]]["image_quality"].values[0]
            context        = df_images.loc[df_images["filename"] == image_infos["file_name"]]["context"].values[0]

            date_time_obj = datetime.datetime.strptime(date_creation, '%Y-%m-%d %H:%M:%S')

            old_filenames.append(image_infos["file_name"])
            dates.append(date_time_obj)
            views.append(view)
            images_quality.append(image_quality)
            contexts.append(context)

            image = Image.open(os.path.join(data_dir,image_infos['file_name']))
            try:
                for orientation in ExifTags.TAGS.keys():
                    if ExifTags.TAGS[orientation]=='Orientation':
                        break
                exif = image._getexif()
                if exif is not None:
                    if exif[orientation] == 3:
                        image=image.rotate(180, expand=True)
                    elif exif[orientation] == 6:
                        image=image.rotate(270, expand=True)
                    elif exif[orientation] == 8:
                        image=image.rotate(90, expand=True)

            except (AttributeError, KeyError, IndexError):
                # cases: image don't have getexif
                pass

            image    = np.array(image) #cv2.cvtColor(np.array(image.convert('RGB')),  cv2.COLOR_RGB2BGR)
            ann_ids  = coco.getAnnIds(imgIds=[img_id])
            anns     = coco.loadAnns(ids=ann_ids)
            h, w     = image.shape[:-1]
            target_h = 1080
            ratio    = target_h/h
            target_w = int(ratio*w) 
            image    = cv2.resize(image,(target_w,target_h))
            h, w     = image.shape[:-1]
            yolo_annot = []
            for ann in anns:
                cat = ann['category_id'] - 1
                [bbox_x, bbox_y, bbox_w, bbox_h] = (ratio*np.array(ann['bbox'])).astype(int)
                bbox = np.array([bbox_x, bbox_y, bbox_w, bbox_h])
                yolo_bbox = coco2yolo(bbox, target_h, target_w)
                yolo_str  = str(cat) + " " + " ".join(yolo_bbox.astype(str))
                yolo_annot.append(yolo_str)
            
            basename  = os.path.splitext(image_infos['file_name'])[0]
            file_name = str(image_infos['id']) + "-" + basename

            img_file_name   = os.path.join("./images", file_name) + ".jpg"
            label_file_name = os.path.join("./labels", file_name) + ".txt"
            
            # Save Label
            with open(label_file_name, 'w') as f:
                f.write('\n'.join(yolo_annot))

            img_to_save = Image.fromarray(image)
            # Save image
            img_to_save.save(img_file_name)

            all_bboxes.append(yolo_annot)
            new_filenames.append(img_file_name)
            new_labelnames.append(label_file_name)
            all_images.append(img_to_save)
            
    my_list = list(zip(old_filenames, dates, views, images_quality, contexts, new_filenames, new_labelnames, all_images, all_bboxes))
    my_df   = pd.DataFrame(my_list, columns=['old_path', 'date', 'view', 'quality','context', 'img_name', 'label_name', 'img', 'bboxes'])
    
    return my_df

df_train = get_df_train_val("annotations-surfnet/instances_train.json")
df_valid = get_df_train_val("annotations-surfnet/instances_val.json")

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

#code to see if using the GPU ; helps for the error cuda but only until 50 epochs

### Group DataFRames for future manipulation

In [ ]:
print(len(df_train))
print(len(df_valid))

In [ ]:
frames = [df_train, df_valid]
df_train_valid = pd.concat(frames)

df_train_valid.head()

In [ ]:
d = df_train_valid.iloc[0]["date"]
def get_day(d):
    d = d.date()
    return int("".join(str(d).split("-")))

df_train_valid["day"] = df_train_valid["date"].apply(lambda x: get_day(x))
df_train_valid.head()

In [ ]:
print(len(df_train_valid))

### Simple Kfold
Cross-validation, evaluate the model on data sample 
k : nb of groups the data splits into ; 
- it shuffles the data; splits it into k groups ; one of them becomes the test and the others become the train sets. We fit the model on the training sets
- we use the mean as a test metric

k is usually 5 or 10

In [ ]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=7)
df_train_valid = df_train_valid.reset_index(drop=True)
df_train_valid['fold'] = -1

for fold, (train_idx, val_idx) in enumerate(kf.split(df_train_valid)):
    df_train_valid.loc[val_idx, 'fold'] = fold

display(df_train_valid.fold.value_counts())

In [ ]:
FOLD = 1

train_files = []
val_files   = []
train_df = df_train_valid.query("fold!=@FOLD")
valid_df = df_train_valid.query("fold==@FOLD")

train_files = list(train_df["img_name"].unique())
val_files   = list(valid_df["img_name"].unique())
len(train_files), len(val_files)

In [ ]:
print("Train", 100 * len(train_files)/(len(train_files)+len(val_files)))
print("Valid", 100 * len(val_files)/(len(train_files)+len(val_files)))

In [ ]:
def plot_image_an_bboxes_yolo(image, annotation_list):
    annotations = np.array(annotation_list)
    w, h = image.size
    
    plotted_image = ImageDraw.Draw(image)

    transformed_annotations = np.copy(annotations)
    transformed_annotations[:,[1,3]] = annotations[:,[1,3]] * w
    transformed_annotations[:,[2,4]] = annotations[:,[2,4]] * h 
    
    transformed_annotations[:,1] = transformed_annotations[:,1] - (transformed_annotations[:,3] / 2)
    transformed_annotations[:,2] = transformed_annotations[:,2] - (transformed_annotations[:,4] / 2)
    transformed_annotations[:,3] = transformed_annotations[:,1] + transformed_annotations[:,3]
    transformed_annotations[:,4] = transformed_annotations[:,2] + transformed_annotations[:,4]
    
    for ann in transformed_annotations:
        obj_cls, x0, y0, x1, y1 = ann
        plotted_image.rectangle(((x0,y0), (x1,y1)), outline="#ff8300", width=5)
        
        plotted_image.text((x0, y0 - 10), class_id_to_name_mapping[(int(obj_cls))])
    
    plt.figure(figsize = (12,10))
    plt.imshow(np.array(image))
    plt.show()

In [ ]:
row        = df_train_valid.iloc[282]
img_name   = os.path.join("images-surfnet/images2label", row["old_path"])
img        = row["img"]
label_list = row["bboxes"]
label_list = [x.split(" ") for x in label_list]
label_list = [[float(y) for y in x ] for x in label_list]

plot_image_an_bboxes_yolo(img, label_list)

## Training

In [ ]:
import yaml

cwd = './'

with open(os.path.join( cwd , 'train.txt'), 'w') as f:
    for path in train_files:
        f.write(path+'\n')
            
with open(os.path.join(cwd , 'val.txt'), 'w') as f:
    for path in val_files:
        f.write(path+'\n')

data = dict(
    path  = './../',
    train =  os.path.join( cwd , 'train.txt') ,
    val   =  os.path.join( cwd , 'val.txt' ),
    nc    = 10,
    names = ['Sheet / tarp / plastic bag / fragment', 'Insulating material', 'Bottle-shaped', 'Can-shaped', 'Drum', 'Other packaging', 'Tire', 'Fishing net / cord', 'Easily namable', 'Unclear'],
    )

with open(os.path.join( cwd , 'data.yaml'), 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

f = open(os.path.join( cwd , 'data.yaml'), 'r')
print('\nyaml:')
print(f.read())

In [ ]:
!git clone https://github.com/ultralytics/yolov5.git

In [ ]:
%%writefile yolov5/data/hyps/hyp.scratch.yaml

# YOLOv5 🚀 by Ultralytics, GPL-3.0 license
# Hyperparameters for COCO training from scratch
# python train.py --batch 40 --cfg yolov5m.yaml --weights '' --data coco.yaml --img 640 --epochs 300
# See tutorials for hyperparameter evolution https://github.com/ultralytics/yolov5#tutorials

lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.1  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.5  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 1.0  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
hsv_h: 0.012  # image HSV-Hue augmentation (fraction)
hsv_s: 0.6  # image HSV-Saturation augmentation (fraction)
hsv_v: 0.3  # image HSV-Value augmentation (fraction)
degrees: 2.0  # image rotation (+/- deg)
translate: 0.3  # image translation (+/- fraction)
scale: 0.25  # image scale (+/- gain)
shear: 0.0  # image shear (+/- deg)
perspective: 0.0  # image perspective (+/- fraction), range 0-0.001
flipud: 0.1  # image flip up-down (probability)
fliplr: 0.5  # image flip left-right (probability)
mosaic: 0.7  # image mosaic (probability)
mixup: 0.0  # image mixup (probability)
copy_paste: 0.0  # segment copy-paste (probability)

A model trained with higher resolution images can have better results for detecting small objects

In [ ]:
!python "yolov5/train.py" --img 640 --hyp "yolov5/data/hyps/hyp.scratch.yaml" --batch 32 --epochs 30 --data "data.yaml" --weights "yolov5s.pt" --workers 23 --project "ben" --name "yolo_ben_30_I&L" --exist-ok 

In [ ]:
ls

In [ ]:
!python "yolov5/train.py" --img 640 --hyp "yolov5/data/hyps/hyp.scratch.yaml" --batch 20 --epochs 30 --data "data.yaml" --weights "yolov5s.pt" --workers 23 --project "ben" --name "yolo_ben_1" --exist-ok 

In [ ]:
!python "yolov5/train.py" --img 640 --hyp "yolov5/data/hyps/hyp.scratch.yaml" --batch 64 --epochs 100 --data "data.yaml" --weights "yolov5s.pt" --workers 23 --project "ben" --name "yolo_ben_100" --exist-ok --device 0

In [ ]:
!python "yolov5/train.py" --img 640 --hyp "yolov5/data/hyps/hyp.scratch.yaml" --batch 32 --epochs 50 --data "data.yaml" --weights "yolov5s.pt" --workers 23 --project "ben" --name "yolo_ben_2" --exist-ok  --device 0

add smthg to device ; une des variables pas dans le GPU 

In [ ]:
!python "yolov5/train.py" --img 640 --hyp "yolov5/data/hyps/hyp.scratch.yaml" --batch 32 --epochs 10 0 --data "data.yaml" --weights "yolov5s.pt" --workers 23 -